# 魔法方法

魔法方法总是被双下划线包围，魔法方法是面向对象的python的一切，它能在适当的时候被调用

### 1.构建以及初始化

_ _new_,_ _init_,_ _del_,_ _prepared_,_ _init_subclass_

In [1]:
#一般我们相对一个函数进行初始化操作时（有传入参数的需求时），加入__init__
class Rectangle:
    def __init__(self,x,y):   #__innit__语句中不能包含return，实际上其return 的值为none
        self.x=x
        self.y=y
    def getPeri(self):
        return (self.x+self.y)*2
    def getArea(self):
        return self.x*self.y

In [2]:
rect=Rectangle(3,4)
print(rect.getPeri())
print(rect.getArea())

14
12


In [3]:
class A:
    def __init__(self):
        return "A"

In [4]:
a=A()

TypeError: __init__() should return None, not 'str'

In [5]:
#实例化对象第一个被调用的魔法方法不是__init__，而是__new__(cls[,...])
#对象是先调用__new__方法，创建一个类的实例，然后将其传递给init方法进行个性化定制
class Capstr(str):  #继承于字符函数
    def __new__(cls,string):
        string=string.upper()  #全部大写，重写操作
        return super().__new__(cls,string) #等同于string.__new__(cls,string)

In [6]:
cs=Capstr("love")
cs

'LOVE'

In [7]:
#父类继承，具有str函数的所有功能
cs.lower()

'love'

In [8]:
cs.capitalize()

'Love'

In [9]:
#__del__(self)

class C:
    def __init__(self):
        print("__init__方法被调用")
    def __del__(self):
        print("__del__方法被调用")

In [12]:
c1=C()  #实例化对象

__init__方法被调用


In [15]:
c2=c1
c3=c2

In [16]:
del c3 
#对象销毁前的最后拦截
#并非发生del时魔法方法__del__就会被调用，所有对对象的引用（即指向对象的所有变量）都被删除后才会被销毁（垃圾回收机制）

In [17]:
del c2

In [18]:
del c1

__del__方法被调用


In [1]:
#在del执行前，将对象传输出来
class D:
    def __init__(self,name):
        self.name=name
    def __del__(self):
        x=self
        print(x)

In [3]:
d=D("python")
d

In [4]:
d.name

'python'

In [5]:
del d

In [6]:
d

NameError: name 'd' is not defined

In [7]:
x

NameError: name 'x' is not defined

In [8]:
#在__del__前把对象传输出来，无法在执行语句的同时去把他的返回值赋值给另一个变量去存储（局部变量）
#方法一：使用全局变量的形式传输出来
class D:
    def __init__(self,name):
        self.name=name
    def __del__(self):
        global x
        x=self

In [9]:
d=D("python")
print(d)
print(d.name)

python


In [10]:
del d

In [11]:
d

NameError: name 'd' is not defined

In [12]:
x

非不得已不使用全局变量，会污染命名空间

In [1]:
#将self（局部变量）用函数的形式传递进去
#python的闭包会通过某种方式将外层函数的局部变量保存下来
#通过闭包的方式让self保存在外部函数的x变量中，内部函数用于窃取self对象
class E:
    def __init__(self,name,func):
        self.name=name
        self.func=func
    def __del__(self):
        self.func(self)   

In [2]:
def outter():
    x=0
    def inner(y=None):
        nonlocal x
        if y:
            x=y
        else:
            return x
    return inner

In [3]:
f=outter()
e=E("XMU",f)

In [4]:
e.name

'XMU'

In [5]:
del e

In [6]:
g=f()

In [7]:
g.name

'XMU'

### 2.运算相关的魔法方法

#### 算术运算

In [19]:
type(len)   #BIF（内置函数）

builtin_function_or_method

In [20]:
type(dir) 

builtin_function_or_method

In [21]:
type(int) 

type

In [22]:
type(list) 

type

In [23]:
class C:
    pass

#C在定义的时候是类，定义完后为类对象

In [24]:
type(C)

type

In [25]:
class new_int(int):
    def __add__(self,other):
        return int.__sub__(self,other)
    def __sub__(self,other):
        return int.__add__(self,other)

In [26]:
a=new_int(3)
b=new_int(5)
a+b

-2

In [27]:
a-b

8

In [1]:
class try_int(int):
    def __add__(self,other):
        return self+other
    def __sub__(self,other):
        return self-other

In [2]:
a=try_int(3)
b=try_int(5)

In [ ]:
a+b   #会出现无限递归，原因是a+b是加法，return之后仍然是加法，会无限循环

In [3]:
#修改错误的代码
class try_int(int):
    def __add__(self,other):
        return int(self)+int(other)
    def __sub__(self,other):
        return int(self)-int(other)

In [4]:
a=try_int(3)
b=try_int(5)

In [5]:
a+b

8

In [36]:
#修改字符串中的加法运算
class S(str):
    def __add__(self,other):
        return len(self)+len(other)

In [37]:
s1=S("Love")
s2=S("Python")
s1+s2

10

In [38]:
s1+"Python"

10

In [39]:
"Love"+s2

'LovePython'

#### 反运算符

__radd__()调用前提：两个对象相加时，如果左侧对象与右侧对象不同类型且左侧对象未定义__add__()方法或者其__add__()返回NotImplemented，那么python就会去右侧的对象中查找是否有__radd__()方法

In [40]:
class Nint(int):
    def __radd__(self,other):
        return int.__sub__(self,other)

In [41]:
a=Nint(5)
b=Nint(3)
a+b

8

In [42]:
1+b 
#python中只能对同类型的对象进行操作，1与b非同类型，add结果为NotImplemnted，只能用b的radd方法，相当于（self）3-（other）1

2

In [30]:
print(type(1))
print(type(b))

<class 'int'>
<class '__main__.Nint'>


In [43]:
b+1
#b和1仍然不是一个类型的，add操作结果返回notimplemented，这时调用1的radd方法 而int类型的radd方法没有被重写，相当于self(1)+other(3)

4

In [5]:
clear

In [6]:
class Nint(int):
    def __rsub__(self,other):
        return int.__sub__(self,other)

In [7]:
a=Nint(5)
3-a     #同理，非同类型的减法只能找a的rsub，a对应self，3对应other

2

#### 增量赋值运算符（运算兼赋值的操作）

In [1]:
class S1(str):
    def __iadd__(self,other):
        return len(self)+len(other)

In [2]:
class S2(str):
    def __radd__(self,other):
        return len(self)+len(other)

In [3]:
s1=S1("Apple")
s2=S2("Banana")
s1+=s2

In [4]:
s1

11

In [5]:
type(s1)

int

如果增强运算符的左侧对象未实现相应的魔法方法，加等于的左侧的对象未实现__iadd__()方法，python会退而求其次，使用相应的__add__()方法和__radd__()方法来替代

In [6]:
s2+=s2 
#左右都是同种类型的对象，不会使用radd方法，而是去父类里面去找__add__()方法

In [7]:
s2

'BananaBanana'

In [8]:
type(s2)

str

In [9]:
s3=S1("python")
s2+=s3

In [10]:
s2

'BananaBananapython'

In [11]:
s3+=s2
s3

24

#### 按位运算

In [48]:
#按位与
3 & 2

2

In [49]:
3 & 4

0

In [50]:
#bin函数获得整数的二进制形式
bin(2)  #0b表示其为二进制字符串

'0b10'

&进行位与计算，只有当相同位的值均为1的情况，2的二进制为010,3的二进制为011，只有第二位同时为1，按位与的结果为二进制的10（十进制的2），3的二进制为011,4的二进制为100，没有相同位为1，按位与计算为二进制的000（十进制的0）

In [51]:
#按位或（管道符）
3 | 2

3

In [52]:
3|4

7

2（010）和3（011）的二进制位只要有一位是1，对应结果的二进制位也是1，按位或的结果为011（十进制的3）；同理，3（011）和4（100）的按位或运算为111（十进制的7）

In [53]:
#按位非（~）：将每个二进制位进行取反减1
~2

-3

In [54]:
~3

-4

In [55]:
~4

-5

补码：在计算机底层对二进制数进行表示运算和存储使用

In [56]:
#按位异或（^），当两个相同的二进制位的值不一样时，结果对应二进制位的值为1
3^2

1

In [57]:
3^4

7

2（010）和3（011）按位异或的结果为（001）也就是1,；3（011）和4（100）按位异或的结果为（111）也就是7
1（001）2（010）3（011）4（100）5（101）6（110）7（111）

In [58]:
#左移右移运算,运算符的左侧为运算的对象，右侧为指定移动的位数
#左移右移运算必须为正数，不能是负数，不然会报错
bin(8)

'0b1000'

In [59]:
8>>2  #去掉了1000右边的两个零

2

In [60]:
8>>3 #去掉了1000右边的3个零

1

In [61]:
8<<2 #右边填充了2个0，变成100000

32

In [62]:
8<<3 #右边填充了三个0，变成1000000

64

右移N位就等于除以2的N次方（地板除取整），左移N位就是乘以2的N次方

优先级：按位或、按位异或、按位与、移位的优先级依次递增，按位非和正号负号处于同一优先级

#### index方法

In [63]:
#__index__
class C:
    def __index__(self):
        print("被拦截")
        return 3

In [64]:
c=C()
c[2]

TypeError: 'C' object is not subscriptable

In [66]:
#当对象作为索引值或者参数的时候，才会触发index方法
s="python"
s[c]

被拦截


'h'

In [67]:
bin(c)

被拦截


'0b11'

### 3.与属性访问相关的函数与魔法方法

对象可以通过.来访问属性，同时还可以通过.创建一个未曾有的属性；Python中专门为对象的属性访问服务的BIF函数：hasattr(),getattr(),setattr(),delattr()

In [71]:
class C:
    def __init__(self,name,age):
        self.name=name 
        self.__age=age  #表示该变量为私有变量
        

In [72]:
c=C("python",100)

In [73]:
hasattr(c,"name")   #判断是否有name这一属性

True

In [74]:
getattr(c,"name")   #获取对象的属性值

'python'

In [75]:
#如果想要获取私有变量的值，需要知道name mangling的规则（名字改编技术）
#需要通过_类名__属性名
getattr(c,"_C__age")

100

In [76]:
setattr(c,"_C__age",101)

In [77]:
getattr(c,"_C__age")

101

In [78]:
delattr(c,"_C__age")

In [79]:
hasattr(c,"_C__age") 

False

魔法方法里面的属性相关：__getattr__,__setattr__,__delattr__

In [38]:
#与getattr函数对应的魔法方法是__getattribute__
class A:
    def __init__(self,name,age):
        self.name=name
        self.__age=age
    def __getattribute__(self,attrname):
        print("拦截")
        return super().__getattribute__(attrname)

In [39]:
a=A("python",100)

In [40]:
getattr(a,"name")

拦截


'python'

In [41]:
a._A__age

拦截


100

In [42]:
a.path

拦截


AttributeError: 'A' object has no attribute 'path'

In [43]:
#在获到对象属性时，__getattribute__()是一定会被调用的，无论属性存不存在，首先都会调用这个魔法方法
#如果调用像a.path这种不存在的对象时，调用__getattribute__()找不到path这个属性，就会再调用__getattr__()这个魔法方法,通过__getattr__()来设置属性不存在时的默认值
class B:
    def __init__(self,name,age):
        self.name=name
        self.__age=age
    def __getattribute__(self,attrname):
        print("拦截")
        return super().__getattribute__(attrname)
    def __getattr__(self,attrname):
        if attrname=="path":
            print("love path")
        else:
            raise AttributeError(attrname)

In [44]:
b=B("python",100)

In [45]:
b.path

拦截
love path


In [46]:
b.x

拦截


AttributeError: x

In [1]:
#__setattr__()
class B:
    def __setattr__(self,name,value):
        self.name=value

In [2]:
b=B()

In [ ]:
b.name="python"  #无限递归报错，重复调用魔法方法

In [1]:
class B:
    def __setattr__(self,name,value):
        self.__dict__[name]=value

In [2]:
b=B()

In [3]:
b.name="python"

In [4]:
b.name

'python'

In [47]:
#__delattr__
class B:
    def __setattr__(self,name,value):
        self.__dict__[name]=value
    def __delattr__(self,name):
        del self.__dict__[name]

In [48]:
b=B()
b.name="python"

In [49]:
b.__dict__

{'name': 'python'}

In [50]:
del b.name

In [51]:
b.__dict__

{}

### 4.对象被索引时，python究竟会怎么做？python会调用一个名为__getitem__(self,index)的魔法方法，这个魔法方法既可以响应单个下标的索引操作，又能支持代表范围的切片索引方式

In [1]:
class C:
    def __getitem__(self,index):
        print(index)

In [2]:
c=C()

In [3]:
c[2]

2


In [4]:
c[2:8]  #slice函数，切片操作相当于是slice函数的语法糖

slice(2, 8, None)


In [5]:
s="i love python"

In [6]:
s[slice(2,6)]

'love'

In [7]:
s[7:]

'python'

In [8]:
s[slice(7,None)]

'python'

In [9]:
s[::4]

'ivyn'

In [10]:
s[slice(None,None,4)]

'ivyn'

In [11]:
#为索引或者切片赋值时，就会被__setitem__()魔法方法所拦截
class D:
    def __init__(self,data):
        self.data=data
    def __getitem__(self,index):
        return self.data[index]
    def __setitem__(self,index,value):
        self.data[index]=value

In [12]:
d=D([1,2,3,4,5])
d[1]

2

In [13]:
d[1]=1

In [14]:
d[1]

1

In [15]:
d[2:4]=[2,3]

In [17]:
d[:]

[1, 1, 2, 3, 5]

In [18]:
class D:
    def __init__(self,data):
        self.data=data
    def __getitem__(self,index):
        return self.data[index]*2

In [20]:
d=D([1,2,3,4,5])
for i in d:  #for语句会访问getitem的魔法方法
    print(i,end=" ")

2 4 6 8 10 

### 5.针对可迭代对象有__iter__和__next__

In [21]:
#__iter__(self)和__next__(self)对应的BIF函数是iter()和next()函数
#对应可迭代对象与迭代器概念
x=[1,2,3,4,5]  #没有__next__函数，不是迭代器
dir(x)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [22]:
next(x)

TypeError: 'list' object is not an iterator

In [23]:
for i in x:   #for语句首先先将对象传入内置函数iter()中，并由此拿到相应的迭代器
    print(i,end=" ")

1 2 3 4 5 

In [25]:
#模拟for语句的形成过程
_=iter(x)   #临时变量
while True:
    try:
        i=_.__next__()
    except StopIteration:
        break
    print(i,end=" ")

1 2 3 4 5 

In [31]:
#制造一个迭代器对象
class Double:
    def __init__(self,start,stop):
        self.value=start-1
        self.stop=stop
    def __iter__(self):#可迭代对象
        return self
    def __next__(self):
        if self.value==self.stop:
            raise StopIteration
        self.value+=1
        return self.value*2

In [32]:
d=Double(1,5)
for i in d:
    print(i,end=" ")

2 4 6 8 10 

https://pyzh.readthedocs.io/en/latest/python-magic-methods-guide.html

#### contains(用于检测成员关系)，对应运算符为in 和not in

In [33]:
class C:
    def __init__(self,data):
        self.data=data
    def __contains__(self,item):
        print("hello")
        return item in self.data

In [34]:
c=C([1,2,3,4,5])
3 in c

hello


True

In [37]:
#python中存在很多代偿的实现，比如在for循环语句中，python在找不到iter和next魔法方法的情况下，就会尝试寻找getitem方法
#python在找不到contains魔法方法的情况下，就会尝试寻找iter和next方法
class C:
    def __init__(self,data):
        self.data=data
    def __iter__(self):   #把参数变成迭代器，然后挨个提取数据
        print("Iter",end="->")
        self.i=0
        return self
    def __next__(self):
        print("Next",end="->")
        if self.i==len(self.data):
            raise StopIteration
        item=self.data[self.i]
        self.i+=1
        print(item)
        return item
    

In [38]:
c=C([1,2,3,4,5])
3 in c

Iter->Next->1
Next->2
Next->3


True

In [35]:
class C:
    def __init__(self,data):
        self.data=data
    def __getitem__(self,index):
        print("GetItem",end="->")
        return self.data[index]

In [36]:
c=C([1,2,3,4,5])
3 in c

GetItem->GetItem->GetItem->

True

In [37]:
#bool函数的代偿是__len__
class D:
    def __init__(self,data):
        self.data=data
    def __len__(self):
        print("len")
        return len(self.data)

In [38]:
d=D("python")
bool(d)    #只要字符串的长度为非零长度就会返回true

len


True

In [39]:
d=D("")
bool(d)

len


False

### 比较运算

In [44]:
'''
__lt__(self,other)<
__le__(self,other)<=
__gt__(self,other)>
__ge__(self,other)>=
__eq__(self,other)==
__ne__(self,other)!=
'''

'\n__lt__(self,other)<\n__le__(self,other)<=\n__gt__(self,other)>\n__ge__(self,other)>=\n__eq__(self,other)==\n__ne__(self,other)!=\n'

In [46]:
class S(str):
    def __lt__(self,other):
        return len(self)<len(other)
    def __gt__(self,other):
        return len(self)>len(other)
    def __eq__(self,other):
        return len(self)==len(other)

In [48]:
#主要集中在对字符的讨论上，之前比较字符比较的是编码但是现在比较的是字符的长度
s1=S("Python")
s2=S("python")
print(s1<s2)
print(s1>s2)
print(s1==s2)

False
False
True


In [49]:
print(s1!=s2)
#通过eq对等值判断进行了拦截，但不意味着不等值判断就会自动取等值判断的相反结果
#由于我们执行s1!=s2时，S类中没有定义，在其父类字符串中寻找不等值判断

True


In [50]:
print(s1<=s2)
print(s1>=s2)  #比较的是编码值的大小

True
False


In [51]:
#若不想让某个魔法方法生效，可直接将其复制为None
class S(str):
    def __lt__(self,other):
        return len(self)<len(other)
    def __gt__(self,other):
        return len(self)>len(other)
    def __eq__(self,other):
        return len(self)==len(other)
    __le__=None
    __ge__=None
    __ne__=None

In [52]:
s1=S("Python")
s2=S("python")
print(s1!=s2)

TypeError: 'NoneType' object is not callable

In [53]:
#出现None时就不会再进行补偿运算
class C:
    def __init__(self,data):
        self.data=data
    def __iter__(self):   #把参数变成迭代器，然后挨个提取数据
        print("Iter",end="->")
        self.i=0
        return self
    def __next__(self):
        print("Next",end="->")
        if self.i==len(self.data):
            raise StopIteration
        item=self.data[self.i]
        self.i+=1
        print(item)
        return item
    __contains__=None

In [54]:
c=C([1,2,3,4,5])
3 in c

TypeError: 'C' object is not a container

### call

In [59]:
class C:
    def __call__(self):
        print("hello")
            

In [61]:
c=C()
c()#call魔法方法被访问

hello


In [62]:
#call支持可变参数和可变关键词参数
class C:
    def __call__(self,*args,**kwargs):
        print(f'位置参数-> {args}\n关键字参数-> {kwargs}')

In [63]:
c=C()
c(1,2,3,x=120,y=996)

位置参数-> (1, 2, 3)
关键字参数-> {'x': 120, 'y': 996}
